In [1]:
pip install datasets transformers faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
i

In [2]:
import faiss
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

In [3]:
device = torch.device('cpu')

In [4]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
print("Loading Wikipedia dataset...")
wiki_dataset = load_dataset('wikipedia', '20220301.en', split='train[:1%]')  # Using only 1% for demonstration

documents = [article['text'] for article in wiki_dataset]

def truncate_document(doc, max_length=1000):
    return doc[:max_length]

# Truncate all documents to a max length
documents = [truncate_document(doc) for doc in documents]

Loading Wikipedia dataset...


The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [6]:
def create_faiss_index(docs):
    print("Creating embeddings...")

    embeddings = embedder.encode(docs, convert_to_tensor=True, device=device)
    embeddings = embeddings.numpy()

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, embeddings

index, doc_embeddings = create_faiss_index(documents)

Creating embeddings...


In [7]:
def retrieve_documents(query, index, documents, k=2):
    query_embedding = embedder.encode(query, convert_to_tensor=True, device=device)
    query_embedding = query_embedding.numpy()

    distances, indices = index.search(query_embedding.reshape(1, -1), k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs

In [12]:
def generate_response(query, retrieved_docs, max_input_length=1024, max_new_tokens=150):
    context = " ".join(retrieved_docs)
    input_text = f"Context: {context}\nUser: {query}\nBot:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    if input_ids.shape[1] > max_input_length:
        input_ids = input_ids[:, -max_input_length:]

    output = gpt_model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1
    )

    # Decoding the generated text
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [13]:
def rag_chatbot(query):
    retrieved_docs = retrieve_documents(query, index, documents)
    response = generate_response(query, retrieved_docs)
    return response

In [16]:
user_query = "Tell me about Fyodor Doestoevsky."
bot_response = rag_chatbot(user_query)
print("Bot:", bot_response)

Bot: Context: Fyodor Mikhailovich Dostoevsky (, ; ; 11 November 18219 February 1881), sometimes transliterated as Dostoyevsky, was a Russian novelist, short story writer, essayist, and journalist. Dostoevsky's literary works explore the human condition in the troubled political, social, and spiritual atmospheres of 19th-century Russia, and engage with a variety of philosophical and religious themes. His most acclaimed novels include Crime and Punishment (1866), The Idiot (1869), Demons (1872), and The Brothers Karamazov (1880). His 1864 novella Notes from Underground is considered to be one of the first works of existentialist literature. Numerous literary critics rate him as one of the greatest novelists in all of world literature, as many of his works are considered highly influential masterpieces.

Born in Moscow in 1821, Dostoevsky was introduced to literature at an early age through fairy tales and legends, and through books by Russian and foreign authors. His mother died in 1837 